# Question 3: VQ-VAE [40pts]
In this question, you with train a [VQ-VAE](https://arxiv.org/abs/1711.00937) on the colored shapes dataset and SVHN. If you are confused on how the VQ-VAE works, you may find [Lilian Weng's blogpost](https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html#vq-vae-and-vq-vae-2) to be useful.

You may experiment with different hyperparameters and architecture designs, but the following designs for the VQ-VAE architecture may be useful.

```
conv2d(in_channels, out_channels, kernel_size, stride, padding)
transpose_conv2d(in_channels, out_channels, kernel_size, stride, padding)
linear(in_dim, out_dim)
batch_norm2d(dim)

residual_block(dim)
    batch_norm2d(dim)
    relu()
    conv2d(dim, dim, 3, 1, 1)
    batch_norm2d(dim)
    relu()
    conv2d(dim, dim, 1, 1, 0)

Encoder
    conv2d(3, 256, 4, 2, 1) 16 x 16
    batch_norm2d(256)
    relu()
    conv2d(256, 256, 4, 2, 1) 8 x 8
    residual_block(256)
    residual_block(256)

Decoder
    residual_block(256)
    residual_block(256)
    batch_norm2d(256)
    relu()
    transpose_conv2d(256, 256, 4, 2, 1) 16 x 16
    batch_norm2d(256)
    relu()
    transpose_conv2d(256, 3, 4, 2, 1) 32 x 32
```

A few other tips:
*   Use a codebook with $K = 128$ latents each with a $D = 256$ dimensional embedding vector
*   You should initialize each element in your $K\times D$ codebook to be uniformly random in $[-1/K, 1/K]$
*   Use batch size 128 with a learning rate of $10^{-3}$ and an Adam optimizer
*   Center and scale your images to $[-1, 1]$
*   Supposing that $z_e(x)$ is the encoder output, and $z_q(x)$ is the quantized output using the codebook, you can implement the straight-through estimator as follows (where below is fed into the decoder): 
  * `(z_q(x) - z_e(x)).detach() + z_e(x)` in Pytorch
  * `tf.stop_gradient(z_q(x) - z_e(x)) + z_e(x)` in Tensorflow.

In addition to training the VQ-VAE, you will also need to train a PixelCNN prior on the categorical latents in order to sample. For your architecture, you may find the following useful:
*   Since the input is a 2D grid of discrete values, you should have an input (learned) embedding layer to map the discrete values to embeddings of length $64$
*   Use a single Type A masked convolutions followed by 10-15 residual blocks (same structure as HW 1), and $2$ $1\times 1$ convolutions of $512$ and $K$ channels respectively.
*   You may find normalization methods (e.g. BatchNorm, LayerNorm) to be useful
*   Use batch size 128 with a learning rate of $10^{-3}$ and an Adam optimizer

**You will provide the following deliverables**


1.   Over the course of training, record the average loss of the training data (per minibatch) and test data (for your entire test set) **for both your VQ-VAE and PixelCNN prior**. Code is provided that automatically plots the training curves. 
2. Report the final test set performances of your final models
3. 100 samples from your trained VQ-VAE and PixelCNN prior
4. 50 real-image / reconstruction pairs (for some $x$, encode and then decode)

In [1]:
import os
import glob
import torch
import math
import numpy as np
import torch.nn.functional as F
from pathlib import Path
from torch import nn, optim
from collections import OrderedDict
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

torch.manual_seed(123)

## Dataset

In [2]:
class FaceImagesDataset(Dataset):
    def __init__(self, path_pattern, transform=None):
        self.transform = transform
        self.root_paths = glob.glob(path_pattern)
    
    def __len__(self):
        return len(self.root_paths)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        image = io.imread(self.root_paths[idx]).astype('float32')
        image = image / 255.
        
        if self.transform:
            image = self.transform(image)
        
        return image

In [3]:
IMAGE_HEIGHT = 124
IMAGE_WIDTH = 108

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH))]
)

train_dataset = FaceImagesDataset('kinship_ver_t1/train-faces/*/MID*/*.jpg', transform=transform)
val_dataset = FaceImagesDataset('kinship_ver_t1/val-faces/*/MID*/*.jpg', transform=transform)
test_dataset = FaceImagesDataset('kinship_ver_t1/test-faces/*.jpg', transform=transform)

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 15845
Validation dataset size: 5045
Test dataset size: 5226


In [4]:
BATCH_SIZE = 16
SHUFFLE = True
NUM_WORKERS = 0
CKPT_PERIOD = 20
LOGS_VQVAE_PATH = './logs/vqvae'
CODE_DIM, CODE_SIZE = 256, 128
MODEL_NAME = 'model_3'
CKPT_VQVAE_PATH = f'./checkpoints/vqvae/{MODEL_NAME}'
Path(CKPT_VQVAE_PATH).mkdir(parents=True, exist_ok=True)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

## General training procedure

In [6]:
from tqdm import tqdm

def train(model, train_loader, optimizer, epoch, quiet, grad_clip=None):
    model.train()

    if not quiet:
        pbar = tqdm(total=len(train_loader.dataset))
    losses = OrderedDict()
    for x in train_loader:
        x = x.cuda()
        out = model.loss(x)
        optimizer.zero_grad()
        out['loss'].backward()
        if grad_clip:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        desc = f'Epoch {epoch}'
        for k, v in out.items():
            if k not in losses:
                losses[k] = []
            losses[k].append(v.item())
            avg_loss = np.mean(losses[k][-50:])
            desc += f', {k} {avg_loss:.4f}'

        if not quiet:
            pbar.set_description(desc)
            pbar.update(x.shape[0])
    if not quiet:
        pbar.close()
    return losses


def eval_loss(model, data_loader, quiet):
    model.eval()
    total_losses = OrderedDict()
    with torch.no_grad():
        for x in data_loader:
            x = x.cuda()
            out = model.loss(x)
            for k, v in out.items():
                total_losses[k] = total_losses.get(k, 0) + v.item() * x.shape[0]

        desc = 'Validation '
        for k in total_losses.keys():
            total_losses[k] /= len(data_loader.dataset)
            desc += f', {k} {total_losses[k]:.4f}'
        if not quiet:
            print(desc)
    return total_losses


def train_epochs(model, train_loader, val_loader, train_args, tb_writer, quiet=False):
    epochs, lr = train_args['epochs'], train_args['lr']
    grad_clip = train_args.get('grad_clip', None)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        train_loss = train(model, train_loader, optimizer, epoch, quiet, grad_clip)
        val_loss = eval_loss(model, val_loader, quiet)
        
        # Save checkpoints
        if epoch % CKPT_PERIOD == 0:
            ckpt_path = f'{CKPT_VQVAE_PATH}/{MODEL_NAME}_{epoch}.pt'
            torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, ckpt_path)
        # Write losses to tensorboard
        for k in train_loss.keys():
            tb_writer.add_scalar(f'training/{k}', np.mean(train_loss[k]), epoch)
            tb_writer.add_scalar(f'validation/{k}', val_loss[k], epoch)

## VQ-VAE and PixelCNN models

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(dim),
            nn.ReLU(),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(),
            nn.Conv2d(dim, dim, 1)
        )
      
    def forward(self, x):
        return x + self.net(x)

In [8]:
class LayerNorm(nn.LayerNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1).contiguous()
        x_shape = x.shape
        x = super().forward(x)
        return x.permute(0, 3, 1, 2).contiguous()

In [9]:
class MaskConv2d(nn.Conv2d):
    def __init__(self, mask_type, *args, conditional_size=None, **kwargs):
        assert mask_type == 'A' or mask_type == 'B'
        super().__init__(*args, **kwargs)
        self.register_buffer('mask', torch.zeros_like(self.weight))
        self.create_mask(mask_type)

        if conditional_size is not None:
            self.cond_op = nn.Linear(conditional_size, self.out_channels)

    def forward(self, input, cond=None):
        out = F.conv2d(input, self.weight * self.mask, self.bias, self.stride,
                       self.padding, self.dilation, self.groups)
        if cond is not None:
            cond = self.cond_op(cond)
            out = out + cond.view(cond.shape[0], self.out_channels, 1, 1)
        return out

    def create_mask(self, mask_type):
        k = self.kernel_size[0]
        self.mask[:, :, :k // 2] = 1
        self.mask[:, :, k // 2, :k // 2] = 1
        if mask_type == 'B':
            self.mask[:, :, k // 2, k // 2] = 1

In [10]:
class PixelCNNResBlock(nn.Module):
    def __init__(self, dim, conditional_size=None):
        super().__init__()
        self.block = nn.ModuleList([
            LayerNorm(dim),
            nn.ReLU(),
            MaskConv2d('B', dim, dim // 2, 1, conditional_size=conditional_size),
            LayerNorm(dim // 2),
            nn.ReLU(),
            MaskConv2d('B', dim // 2, dim // 2, 3, padding=1, 
                       conditional_size=conditional_size),
            LayerNorm(dim // 2),
            nn.ReLU(),
            MaskConv2d('B', dim // 2, dim, 1, conditional_size=conditional_size)
        ])

    def forward(self, x, cond=None):
        out = x
        for layer in self.block:
            if isinstance(layer, MaskConv2d):
                out = layer(out, cond=cond)
            else:
                out = layer(out)
        return x + out

In [11]:
class PixelCNN(nn.Module):
    def __init__(self, input_shape, code_size, dim=256, n_layers=7,
                 conditional_size=None):
        super().__init__()
        self.embedding = nn.Embedding(code_size, dim)
        model = nn.ModuleList([MaskConv2d('A', dim, dim, 7, padding=3,
                                          conditional_size=conditional_size),
                               LayerNorm(dim), nn.ReLU()])
        for _ in range(n_layers - 1):
            model.append(PixelCNNResBlock(dim, conditional_size=conditional_size))
        model.extend([LayerNorm(dim), nn.ReLU(), MaskConv2d('B', dim, 512, 1, conditional_size=conditional_size),
                      nn.ReLU(), MaskConv2d('B', 512, code_size, 1,
                                            conditional_size=conditional_size)])
        self.net = model
        self.input_shape = input_shape
        self.code_size = code_size

    def forward(self, x, cond=None):
        print(self.embedding.weight.shape)
        out = self.embedding(x).permute(0, 3, 1, 2).contiguous()
        for layer in self.net:
            if isinstance(layer, MaskConv2d) or isinstance(layer, PixelCNNResBlock):
                out = layer(out, cond=cond)
            else:
                out = layer(out)
        return out

    def loss(self, x, cond=None):
        out = self(x, cond=cond)
        return OrderedDict(loss=F.cross_entropy(out, x))

    def sample(self, n, cond=None):
        samples = torch.zeros(n, *self.input_shape).long().cuda()
        with torch.no_grad():
            for r in range(self.input_shape[0]):
                for c in range(self.input_shape[1]):
                    logits = self(samples, cond=cond)[:, :, r, c]
                    logits = F.softmax(logits, dim=1)
                    samples[:, r, c] = torch.multinomial(logits, 1).squeeze(-1)
        return samples

In [12]:
class Quantize(nn.Module):

    def __init__(self, size, code_dim):
        super().__init__()
        self.embedding = nn.Embedding(size, code_dim)
        self.embedding.weight.data.uniform_(-1./size,1./size)

        self.code_dim = code_dim
        self.size = size

    def forward(self, z):
        b, c, h, w = z.shape
        weight = self.embedding.weight

        flat_inputs = z.permute(0, 2, 3, 1).contiguous().view(-1, self.code_dim)
        
        distances = ((flat_inputs.repeat(1, self.size).view(-1, self.size, self.code_dim) \
                     - weight) ** 2).sum(dim=-1)
        encoding_indices = torch.min(distances, dim=-1)[1]
        encoding_indices = encoding_indices.view(b, h, w)
        quantized = self.embedding(encoding_indices).permute(0, 3, 1, 2).contiguous()

        return quantized, (quantized - z).detach() + z, encoding_indices

In [13]:
class VectorQuantizedVAE(nn.Module):
    def __init__(self, code_dim, code_size):
        super().__init__()
        self.code_size = code_size
        self.code_dim = code_dim

        self.encoder = nn.Sequential(
            nn.Conv2d(3, self.code_dim // 4, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim // 4),
            nn.Conv2d(self.code_dim // 4, self.code_dim // 2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim // 2),
            nn.Conv2d(self.code_dim // 2, self.code_dim, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim),
            nn.Conv2d(self.code_dim, self.code_dim, 4, stride=2, padding=1),
            ResidualBlock(self.code_dim),
            ResidualBlock(self.code_dim),
        )

        self.codebook = Quantize(code_size, code_dim)

        self.decoder = nn.Sequential(
            ResidualBlock(self.code_dim),
            ResidualBlock(self.code_dim),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim),
            nn.ConvTranspose2d(self.code_dim, self.code_dim, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim),
            nn.ConvTranspose2d(self.code_dim, self.code_dim // 2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim // 2),
            nn.ConvTranspose2d(self.code_dim // 2, self.code_dim // 4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(self.code_dim // 4),
            nn.ConvTranspose2d(self.code_dim // 4, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )

    def encode_code(self, x):
        with torch.no_grad():
            x = 2 * x - 1
            z = self.encoder(x)
            indices = self.codebook(z)[2]
            return indices

    def decode_code(self, latents):
        with torch.no_grad():
            latents = self.codebook.embedding(latents).permute(0, 3, 1, 2).contiguous()
            return self.decoder(latents) * 0.5 + 0.5

    def forward(self, x):
        z = self.encoder(x)
        e, e_st, _ = self.codebook(z)
        x_tilde = self.decoder(e_st)

        diff1 = torch.mean((z - e.detach()) ** 2)
        diff2 = torch.mean((e - z.detach()) ** 2)
        return x_tilde, diff1 + diff2

    def loss(self, x):
        x = 2 * x - 1
        x_tilde, diff = self(x)
        recon_loss = F.mse_loss(x_tilde, x)
        loss = recon_loss + diff
        return OrderedDict(loss=loss, recon_loss=recon_loss, reg_loss=diff)

## Train VQ-VAE model

In [14]:
# Instantiate model
vqvae = VectorQuantizedVAE(CODE_DIM, CODE_SIZE).cuda()

# Define TB writer
writer = SummaryWriter(f'{LOGS_VQVAE_PATH}/{MODEL_NAME}')
writer.add_graph(vqvae, train_dataset[0].unsqueeze(0).cuda())
train_epochs(vqvae, train_dataloader, val_dataloader, \
             dict(epochs=500, lr=1e-4, grad_clip=1), writer, quiet=False)
writer.close()

Epoch 0, loss 0.1991, recon_loss 0.0210, reg_loss 0.1782: 100%|██████████| 15845/15845 [02:08<00:00, 123.28it/s]
Epoch 1, loss 0.1450, recon_loss 0.0364, reg_loss 0.1086:   0%|          | 16/15845 [00:00<02:01, 130.53it/s]

Validation , loss 0.1602, recon_loss 0.0151, reg_loss 0.1451


Epoch 1, loss 0.0981, recon_loss 0.0167, reg_loss 0.0813: 100%|██████████| 15845/15845 [02:00<00:00, 131.06it/s]
Epoch 2, loss 0.0982, recon_loss 0.0218, reg_loss 0.0764:   0%|          | 16/15845 [00:00<02:00, 131.29it/s]

Validation , loss 0.0826, recon_loss 0.0126, reg_loss 0.0700


Epoch 2, loss 0.0595, recon_loss 0.0144, reg_loss 0.0451: 100%|██████████| 15845/15845 [02:00<00:00, 131.61it/s]
Epoch 3, loss 0.0749, recon_loss 0.0178, reg_loss 0.0570:   0%|          | 16/15845 [00:00<02:00, 131.34it/s]

...

Validation , loss 0.0080, recon_loss 0.0038, reg_loss 0.0042


Epoch 359, loss 0.0083, recon_loss 0.0038, reg_loss 0.0045: 100%|██████████| 15845/15845 [02:00<00:00, 131.39it/s]
Epoch 360, loss 0.0070, recon_loss 0.0034, reg_loss 0.0037:   0%|          | 16/15845 [00:00<02:00, 131.37it/s]

Validation , loss 0.0084, recon_loss 0.0039, reg_loss 0.0045


Epoch 360, loss 0.0081, recon_loss 0.0036, reg_loss 0.0045:  14%|█▍        | 2272/15845 [00:17<01:43, 131.16it/s]

KeyboardInterrupt: 

In [ ]:
def test_reconstruction(vqvae, batch_cnt = 2):
    test_iterator = iter(test_dataloader)
    x = [next(test_iterator).cuda() for i in range(batch_cnt)]
    x = torch.cat(x, 0)
    with torch.no_grad():
        z = vqvae.encode_code(x)
        x_recon = vqvae.decode_code(z)
    reconstructions = torch.cat((x, x_recon), axis=0)
    return reconstructions

In [ ]:
# Reconstruct some images from test dataset
recs = test_reconstruction(vqvae)
grid_img = make_grid(recs, nrow=recs.shape[0]//2)
plt.figure(figsize = (20, 40))
plt.imshow(grid_img.permute(1, 2, 0).cpu().numpy())

## Train PixelCNN model

In [ ]:
checkpoint = torch.load(PATH)
model = VectorQuantizedVAE(CODE_DIM, CODE_SIZE).cuda()
model.load_state_dict(checkpoint['model_state_dict'])

def create_prior_dataset(data_loader, model):
        prior_data = []
        with torch.no_grad():
            for x in data_loader:
                x = x.cuda()
                z = model.cuda().encode_code(x)
                prior_data.append(z.long())
        return torch.cat(prior_data, dim=0)
    
prior = PixelCNN(code_size=CODE_SIZE, input_shape=(31, 27), dim=128, n_layers=10).cuda()
prior_train_data, prior_val_data = create_prior_dataset(train_dataloader, model), create_prior_dataset(val_dataloader, model)
prior_train_loader = DataLoader(prior_train_data, batch_size=128, shuffle=True)
prior_val_loader = DataLoader(prior_val_data, batch_size=128)
writer = SummaryWriter(f'test')
train_epochs(prior, prior_train_loader, prior_val_loader, \
             dict(epochs=1, lr=1e-3, grad_clip=1), writer, quiet=False)

In [ ]:
def q3(train_data, test_data, dset_id):
    """
    train_data: An (n_train, 32, 32, 3) uint8 numpy array of color images with values in [0, 255]
    test_data: An (n_test, 32, 32, 3) uint8 numpy array of color images with values in [0, 255]
    dset_id: An identifying number of which dataset is given (1 or 2). Most likely
               used to set different hyperparameters for different datasets

    Returns
    - a (# of training iterations,) numpy array of VQ-VAE train losess evaluated every minibatch
    - a (# of epochs + 1,) numpy array of VQ-VAE train losses evaluated once at initialization and after each epoch
    - a (# of training iterations,) numpy array of PixelCNN prior train losess evaluated every minibatch
    - a (# of epochs + 1,) numpy array of PixelCNN prior train losses evaluated once at initialization and after each epoch
    - a (100, 32, 32, 3) numpy array of 100 samples (an equal number from each class) with values in {0, ... 255}
    - a (100, 32, 32, 3) numpy array of 50 real image / reconstruction pairs
      FROM THE TEST SET with values in [0, 255]
    """

    """ YOUR CODE HERE """
    train_data = (np.transpose(train_data, (0, 3, 1, 2)) / 255).astype('float32')
    test_data = (np.transpose(test_data, (0, 3, 1, 2)) / 255).astype('float32')

    code_dim, code_size = 256, 128
    vqvae = VectorQuantizedVAE(code_dim, code_size).cuda()
    train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=128)
    train_losses, test_losses = train_epochs(vqvae, train_loader, test_loader,
                                             dict(epochs=2, lr=1e-3, grad_clip=1), quiet=False)
    vqvae_train_losses, vqvae_test_losses = train_losses['loss'], test_losses['loss']

    def create_prior_dataset(data_loader):
        prior_data = []
        with torch.no_grad():
            for x in data_loader:
                x = x.cuda()
                z = vqvae.encode_code(x)
                prior_data.append(z.long())
        return torch.cat(prior_data, dim=0)
    
    prior = PixelCNN(code_size=code_size, input_shape=(8, 8), dim=128, n_layers=10).cuda()
    prior_train_data, prior_test_data = create_prior_dataset(train_loader), create_prior_dataset(test_loader)
    prior_train_loader = DataLoader(prior_train_data, batch_size=128, shuffle=True)
    prior_test_loader = DataLoader(prior_test_data, batch_size=128)
    prior_train_losses, prior_test_losses = train_epochs(prior, prior_train_loader, prior_test_loader,
                                                         dict(epochs=1, lr=1e-3, grad_clip=1), quiet=False)
    prior_train_losses, prior_test_losses = prior_train_losses['loss'], prior_test_losses['loss']

    samples = prior.sample(100).long()
    samples = vqvae.decode_code(samples) * 255

    x = next(iter(test_loader))[:50].cuda()
    with torch.no_grad():
        z = vqvae.encode_code(x)
        x_recon = vqvae.decode_code(z)
    x = x.cpu().permute(0, 2, 3, 1).numpy()
    reconstructions = np.stack((x, x_recon), axis=1).reshape((-1, 32, 32, 3)) * 255

    return vqvae_train_losses, vqvae_test_losses, prior_train_losses, prior_test_losses, samples, reconstructions